In [2]:
import pandas as pd
import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import scipy.stats as stats

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


RESPONSE_IN_DAYS = 14

In [3]:
df = pd.read_pickle('/home/youlyu2/model_explore/2a_wtext_wbart.pkl')

In [4]:
df.head()

,author,msg,timestamp,contentText,concern_bart-phillipine,concern_bart-china,concern_bart-united states,concern_bart-refugees,concern_bart-defense,concern_bart-economy,concern_bart-economic sactions,concern_bart-energy
0,1757901,1757903,1672534632,@UnkleFlip @TSingletonSays @SportsVix O shit…d...,0.031723,0.000889,0.061526,0.011795,0.084146,0.209006,0.244000,0.256953
1,1757926,1757927,1672534518,"@GrantCardone Oops, my finger hit CDC it shoul...",0.008754,0.000213,0.006890,0.000416,0.006842,0.063662,0.405183,0.000345
2,1757943,1757945,1672534447,RT @timand2037 RT. #Türkiye has agreed to full...,0.121434,0.001188,0.004340,0.013564,0.485215,0.098772,0.024835,0.058622
3,1757972,1757974,1672533731,RT @ItsDanSheehan You say NPC like it's a bad ...,0.061782,0.000837,0.044490,0.003430,0.283441,0.265223,0.211866,0.042541
4,1758002,1758004,1672533679,"RT @libbyliberalnyc ""(Shittin' On) The Dock of...",0.010799,0.000186,0.005134,0.007702,0.238561,0.012402,0.021285,0.002201


In [5]:
maxium_timestamp = max(df["timestamp"])
minium_timestamp = min(df["timestamp"])

In [6]:
print(len(df["author"].unique()))
print(len(df))

1897426
4726147


In [7]:
# Assuming df is your DataFrame and 'author' is the column you want to group by
cols_to_exclude = ['author', 'msg', 'timestamp', 'contentText']
cols_to_average = [col for col in df.columns if col not in cols_to_exclude]

# Define the aggregation dictionary.
agg_dict = {col: ['mean', 'var'] for col in cols_to_average}
agg_dict.update({col: list for col in ['msg', 'timestamp', 'contentText']})

# Group by 'author' and apply the aggregation.
df_grouped = df.groupby("author").agg(agg_dict).reset_index()
# convert the timestamp to days

In [8]:
df_grouped.head()



author concern_bart-phillipine           concern_bart-china                \
                              mean       var               mean           var   
0  1757901                0.066514  0.003222           0.013453  3.007292e-04   
1  1757926                0.182007  0.030654           0.149040  1.040026e-01   
2  1757943                0.080488  0.003353           0.000781  3.318020e-07   
3  1757972                0.072310  0.000222           0.007573  9.074379e-05   
4  1758002                0.153266  0.027116           0.266926  1.383861e-01   

  concern_bart-united states           concern_bart-refugees            \
                        mean       var                  mean       var   
0                   0.130951  0.010999              0.012779  0.000293   
1                   0.148360  0.034973              0.002006  0.000005   
2                   0.017938  0.000370              0.006993  0.000086   
3                   0.054546  0.000202              0.002305  0.000003   
4                   0.379566  0.113753              0.020476  0.002605   

  concern_bart-defense           concern_bart-economy            \
                  mean       var                 mean       var   
0             0.149289  0.048778             0.234368  0.095658   
1             0.093940  0.028667             0.045916  0.002885   
2             0.468503  0.000559             0.194259  0.018236   
3             0.185773  0.019078             0.188284  0.011839   
4             0.246073  0.068771             0.166742  0.051476   

  concern_bart-economic sactions           concern_bart-energy            \
                            mean       var                mean       var   
0                       0.193397  0.043861            0.225735  0.061949   
1                       0.089423  0.031405            0.094107  0.026344   
2                       0.140275  0.026653            0.032721  0.001342   
3                       0.112568  0.019720            0.249627  0.085769   
4                       0.175894  0.054029            0.188158  0.071004   

                                                 msg  \
                                                list   
0  [1757903, 2928450, 4925591, 6055239, 6635574, ...   
1      [1757927, 4868849, 6580468, 6743147, 8015383]   
2                                 [1757945, 2926806]   
3                                 [1757974, 3170546]   
4  [1758004, 1878079, 1950055, 2019728, 2019730, ...   

                                           timestamp  \
                                                list   
0  [1672534632, 1675389971, 1679543516, 168212254...   
1  [1672534518, 1679438173, 1683304550, 168370455...   
2                           [1672534447, 1675319788]   
3                           [1672533731, 1675868782]   
4  [1672533679, 1672795350, 1672953089, 167315598...   

                                         contentText  
                                                list  
0  [@UnkleFlip @TSingletonSays @SportsVix O shit…...  
1  [@GrantCardone Oops, my finger hit CDC it shou...  
2  [RT @timand2037 RT. #Türkiye has agreed to ful...  
3  [RT @ItsDanSheehan You say NPC like it's a bad...  
4  [RT @libbyliberalnyc "(Shittin' On) The Dock o...

In [9]:
def calculate_rate_respond_in_2_weeks(timestamps, current_time=maxium_timestamp):
    
    # Recency: Time difference from the last message to the current time

    latest_message = timestamps[-1]
    target = (RESPONSE_IN_DAYS * 60 * 60 * 24) + current_time - latest_message

    # Consistency: Standard deviation of time intervals between messages
    if len(timestamps) > 2:
        intervals = [(timestamps[i] - timestamps[i-1]) for i in range(1, len(timestamps))]
        std = np.std(intervals)
        mean = np.mean(intervals)
        probability = stats.norm.cdf(target, mean, std) - stats.norm.cdf(0, mean, std)
        return probability
    else:
        return len(timestamps) * RESPONSE_IN_DAYS / ((maxium_timestamp - minium_timestamp) / (60 * 60 * 24))

    # Normalize and combine the metrics
    # Note: You may need to adjust the normalization method based on your data range

In [10]:
df_grouped["response_rate"] = df_grouped[("timestamp", "list")].apply(calculate_rate_respond_in_2_weeks)

In [11]:
df_grouped

author concern_bart-phillipine           concern_bart-china  \
                                    mean       var               mean   
0        1757901                0.066514  0.003222           0.013453   
1        1757926                0.182007  0.030654           0.149040   
2        1757943                0.080488  0.003353           0.000781   
3        1757972                0.072310  0.000222           0.007573   
4        1758002                0.153266  0.027116           0.266926   
...          ...                     ...       ...                ...   
1897421  8420613                0.096156       NaN           0.632033   
1897422  8420617                0.025005       NaN           0.001631   
1897423  8420627                0.129598       NaN           0.982065   
1897424  8420630                0.058618       NaN           0.022858   
1897425  8420635                0.133338       NaN           0.041642   

                      concern_bart-united states            \
                  var                       mean       var   
0        3.007292e-04                   0.130951  0.010999   
1        1.040026e-01                   0.148360  0.034973   
2        3.318020e-07                   0.017938  0.000370   
3        9.074379e-05                   0.054546  0.000202   
4        1.383861e-01                   0.379566  0.113753   
...               ...                        ...       ...   
1897421           NaN                   0.003614       NaN   
1897422           NaN                   0.048630       NaN   
1897423           NaN                   0.000246       NaN   
1897424           NaN                   0.006751       NaN   
1897425           NaN                   0.577544       NaN   

        concern_bart-refugees           concern_bart-defense  ...  \
                         mean       var                 mean  ...   
0                    0.012779  0.000293             0.149289  ...   
1                    0.002006  0.000005             0.093940  ...   
2                    0.006993  0.000086             0.468503  ...   
3                    0.002305  0.000003             0.185773  ...   
4                    0.020476  0.002605             0.246073  ...   
...                       ...       ...                  ...  ...   
1897421              0.000585       NaN             0.111489  ...   
1897422              0.000996       NaN             0.048252  ...   
1897423              0.001247       NaN             0.507368  ...   
1897424              0.313328       NaN             0.798398  ...   
1897425              0.006519       NaN             0.032578  ...   

        concern_bart-economy           concern_bart-economic sactions  \
                        mean       var                           mean   
0                   0.234368  0.095658                       0.193397   
1                   0.045916  0.002885                       0.089423   
2                   0.194259  0.018236                       0.140275   
3                   0.188284  0.011839                       0.112568   
4                   0.166742  0.051476                       0.175894   
...                      ...       ...                            ...   
1897421             0.013177       NaN                       0.004109   
1897422             0.000944       NaN                       0.000271   
1897423             0.596627       NaN                       0.835007   
1897424             0.107698       NaN                       0.254286   
1897425             0.581243       NaN                       0.170562   

                  concern_bart-energy            \
              var                mean       var   
0        0.043861            0.225735  0.061949   
1        0.031405            0.094107  0.026344   
2        0.026653            0.032721  0.001342   
3        0.019720            0.249627  0.085769   
4        0.054029            0.188158  0.071004   
...           ...                 ...       ... 

In [12]:
data = df_grouped
sett = set()

for col, subcol in data.columns:
    if subcol != "":
        sett.add(col)
        data[f"{col}-{subcol}"] = data[col][subcol]

for col in sett:
    del data[col]

data["total_num_msg"] = data["msg-list"].apply(lambda x: len(x))

data.drop(columns=["msg-list", "timestamp-list", "contentText-list"], inplace=True)

data.head()

In [ ]:
len(df_grouped[df_grouped["total_num_msg"] > 50])

4569

In [ ]:
df_more_than_5 = df_grouped[df_grouped["total_num_msg"] > 50]
df_more_than_5.columns = df_more_than_5.columns.get_level_values(0)#.difference(['author', 'response_rate'])
df_more_than_5[df_more_than_5["response_rate"] == max(df_more_than_5["response_rate"])]

,author,response_rate,concern_bart-phillipine-mean,concern_bart-phillipine-var,concern_bart-china-mean,concern_bart-china-var,concern_bart-united states-mean,concern_bart-united states-var,concern_bart-refugees-mean,concern_bart-refugees-var,concern_bart-defense-mean,concern_bart-defense-var,concern_bart-economy-mean,concern_bart-economy-var,concern_bart-economic sactions-mean,concern_bart-economic sactions-var,concern_bart-energy-mean,concern_bart-energy-var,total_num_msg
55399,1918172,0.905715,0.135883,0.008639,0.24513,0.095451,0.321078,0.09313,0.030526,0.008482,0.315801,0.092918,0.304392,0.082021,0.263531,0.085988,0.135733,0.024181,51


In [42]:

X = df_more_than_5[df_more_than_5.columns.difference(['author', 'response_rate'])]
y = df_more_than_5['response_rate']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree Regressor
dt_regressor = RandomForestRegressor(random_state=42)

# Train the model using the training sets
dt_regressor.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = dt_regressor.predict(X_test)

# Calculate metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

# You can also visualize the tree if you want (optional)
# from sklearn.tree import plot_tree
# import matplotlib.pyplot as plt
# plt.figure(figsize=(20,10))  # Set the size of the figure
# plot_tree(dt_regressor, 
#           filled=True, 
#           feature_names=X_train.columns, 
#           class_names=True, 
#           rounded=True, 
#           proportion=False, 
#           precision=2, 
#           fontsize=12)
# plt.show()

Mean Squared Error: 0.0030226993645422703
Mean Absolute Error: 0.04157018348885256
Root Mean Squared Error: 0.05497908115403776
R-squared: 0.4719966827443819
